# 决策树

## ID3

In [8]:
import numpy as np
from collections import Counter

![](./Figures/Algorithm/ID3.png)

In [21]:
class ID3DecisionTree:
    def __init__(self):
        self.tree = None  # 储存最终的决策树

    def _calc_entropy(self, y):
        """计算信息熵"""
        label_counts = Counter(y)
        """
        # 示例：统计列表中元素的出现次数
        fruits = ['苹果', '香蕉', '苹果', '橙子', '香蕉', '苹果', '苹果']
        fruit_counter = Counter(fruits)
        print(fruit_counter)  # 输出：Counter({'苹果': 4, '香蕉': 2, '橙子': 1})
        """
        entropy = 0
        total = sum(label_counts.values())
        for count in label_counts.values():
            prob = count / total
            entropy -= prob * np.log2(prob + 1e-8)  # 避免log(10)

        return entropy

    def _calc_info_gain(self, X, y, feature_idx):
        """计算某个特征的信息增益"""
        base_entropy = self._calc_entropy(y)  # 原始熵
        feature_values = X[:, feature_idx]
        value_count = Counter(feature_values)
        new_entropy = 0.0
        total = len(y)

        # 计算条件熵
        for value, count in value_count.items():
            mask = (feature_values == value)
            # 选出特征X_i == value 的子集
            subset_y = y[mask]
            prob = count / total
            new_entropy += prob * self._calc_entropy(subset_y)

        return base_entropy - new_entropy  # 信息增益 = 原始熵 - 条件熵

    def _choose_best_feature(self, X, y):
        """选择信息增益最大的特征"""
        n_features = X.shape[1]
        best_gain = -1
        best_feature_idx = -1
        for idx in range(n_features):
            gain = self._calc_info_gain(X, y, idx)
            if gain > best_gain:
                best_gain = gain
                best_feature_idx = idx

        return best_feature_idx

    def _majority_vote(self, y):
        """当所有特征都用完时，返回出现次数最多的标签（投票）"""
        """
        y : ['是', '是', '否', '是']
        Counter(y).most_common(1) : [('是', 3)]
        Counter(y).most_common(1)[0][0] : 是
        """
        return Counter(y).most_common(1)[0][0]

    def _build_tree(self, X, y, feature_names):
        """递归构建决策树"""
        # 1. 所有样本标签相同，直接返回该标签
        if len(set(y)) == 1:
            return y[0]
        # 2. 没有特征可划分，返回多数投票结果
        if X.shape[1] == 0:
            return self._majority_vote(y)

        # 3. 选择最优特征
        best_feature_idx = self._choose_best_feature(X, y)
        best_feature_name = feature_names[best_feature_idx]

        # 4. 构建树节点
        tree = {best_feature_name: {}}
        # 删除已使用的特征名
        remaining_feature_names = feature_names[:best_feature_idx] + feature_names[best_feature_idx + 1:]
        # 获取该特征的所有唯一值
        feature_values = X[:, best_feature_idx]
        unique_values = set(feature_values)

        # 5. 递归划分每个特征值对应的子集
        for value in unique_values:
            mask = (feature_values == value)
            subset_x = X[mask, :]
            # 删除已使用的特征列
            subset_x = np.delete(subset_x, best_feature_idx, axis=1)
            subset_y = y[mask]
            # 递归构建子树
            tree[best_feature_name][value] = self._build_tree(subset_x, subset_y, remaining_feature_names)

        return tree

    def fit(self, X, y, feature_names):
        """训练模型：输入特征矩阵X、标签y、特征名称列表"""
        self.tree = self._build_tree(X, y, feature_names)

    def _predict_single(self, x, tree, feature_names):
        """预测单个样本"""
        # 递归遍历树
        root_name = list(tree.keys())[0]
        root_dict = tree[root_name]
        feature_idx = feature_names.index(root_name)
        x_value = x[feature_idx]

        # 找到当前特征值对应的子树
        if x_value in root_dict:
            subtree = root_dict[x_value]
            # 如果子树是叶子节点（标签），返回；否则继续递归
            if isinstance(subtree, dict):
                return self._predict_single(x, subtree, feature_names)
            else:
                return subtree
        else:
            # 若遇到未见过的特征值，返回多数投票（此处简化为返回第一个叶子标签）
            return list(root_dict.values())[0]

    def predict(self, X, feature_names):
        """预测多个样本"""
        return np.array([self._predict_single(x, self.tree, feature_names) for x in X])


In [22]:
# 测试代码
if __name__ == "__main__":
    # 手动构造数据集：是否出去玩的决策（特征：天气、温度、湿度、风速）
    X = np.array([
        ['晴', '热', '高', '弱'],
        ['晴', '热', '高', '强'],
        ['阴', '热', '高', '弱'],
        ['雨', '适温', '高', '弱'],
        ['雨', '凉', '正常', '弱'],
        ['雨', '凉', '正常', '强'],
        ['阴', '凉', '正常', '强'],
        ['晴', '适温', '高', '弱'],
        ['晴', '凉', '正常', '弱'],
        ['雨', '适温', '正常', '弱'],
        ['晴', '适温', '正常', '强'],
        ['阴', '适温', '高', '强'],
        ['阴', '热', '正常', '弱'],
        ['雨', '适温', '高', '强']
    ])
    y = np.array(['否', '否', '是', '是', '是', '否', '是', '否', '是', '是', '是', '是', '是', '否'])
    feature_names = ['天气', '温度', '湿度', '风速']

    # 训练模型
    model = ID3DecisionTree()
    model.fit(X, y, feature_names)
    print("构建的决策树：")
    import json
    print(json.dumps(model.tree, ensure_ascii=False, indent=2))

    # 预测新样本（天气=晴，温度=凉，湿度=正常，风速=弱）
    new_sample = np.array([['晴', '凉', '正常', '弱']])
    prediction = model.predict(new_sample, feature_names)
    print("\n新样本预测结果：", prediction[0])  # 输出：是

构建的决策树：
{
  "天气": {
    "雨": {
      "风速": {
        "强": "否",
        "弱": "是"
      }
    },
    "阴": "是",
    "晴": {
      "湿度": {
        "高": "否",
        "正常": "是"
      }
    }
  }
}

新样本预测结果： 是


构建的决策树：
{
  "天气": {
    "雨": {
      "风速": {
        "强": "否",
        "弱": "是"
      }
    },
    "阴": "是",
    "晴": {
      "湿度": {
        "高": "否",
        "正常": "是"
      }
    }
  }
}

新样本预测结果： 是